# IST256 Project Deliverable 3 (P3)

## Phase 3: Implementation

In this step, you submit the final version of working code. No changes to your code will be considered after this submission. It is important to take prior instructor feedback taken into consideration and these factor into your evaluation.

**IMPORTANT**: Don't forget to journal your work on the project as it factors into the evaluation of your work!

### Step 1: What is Your Idea, Again?

Please reiterate your project idea below (you can copy it from P1/P2).

`--== Double-click and put the title or brief description of your project below  ==--`

My project idea will source country data from multiple websites and then allow you to compare country data across pandas dataframes from multiple sources. The user will be able to select two countries they want to compare, the data they wish to compare, and the source in which to compare from. There are services on the web that allow you to do this, but not an open source app that allows comparison from multiple sources.

This project is relevant to my major and my interests in exploring Pandas further, so far my favorite part of IST 256. I hope to be able to present this program to future employers as a demonstration of my coding skills.

I plan on using HTML source code from chrome to source tables or APIs to source my data. I will pass the data into data tables using Pandas, and then using Interact and if statements I will display relevant tables and information.

### Step 2: Project Code

Include all project code below. Make sure to execute your code to ensure it runs properly before you turn it in. 


In [1]:
!pip install pycountry
!pip install wbdata
!pip install pandas-datareader
from pandas_datareader import wb
import pandas as pd
from IPython.display import display, HTML, display_html
import requests
from itertools import chain, cycle
import numpy as np
from ipywidgets import interact_manual, Dropdown, interact, widgets
import wbdata
import pycountry
import plotly.express as px
import plotly.graph_objects as go
import warnings

     |████████████████████████████████| 10.1 MB 7.2 MB/s eta 0:00:01
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746864 sha256=53b89b2e1900d3e433e45fb9905beb2911d38368acb99574b760c043eb800bc2
  Stored in directory: /home/jovyan/.cache/pip/wheels/09/eb/0d/4ee773c6a4aadc2a43cb5c1d07f268f13c4cdc0eec88e7c1ef
Successfully built pycountry
     |████████████████████████████████| 107 kB 10.1 MB/s eta 0:00:01


In [3]:
# Step 2: Write code here
warnings.filterwarnings('ignore') # blocks warning code flags from pandas
indicators_lookup = {} # empty lsit for naming dataframe headings

# getTopics populates a list of World Bank Data indicators using the function wbdata.get_topic()
def getTopics():
    topics = []
    topic_list = wbdata.get_topic()
    for topic in topic_list:
        topics.append([topic['value'], topic['id']])
    return topics

# getIndicators populates a list of indicators based off of the topic chosen in the interact dropdown
def getIndicators(topicID):
    indicators = []
    indicator_list = wbdata.get_indicator(topic = topicID)
    for indicator in indicator_list:
        indicators.append([indicator['name'], indicator['id']])
        indicators_lookup[indicator['id']] = indicator['name']
    return indicators

# getCountries uses the module pycountries to populate a list of country full names and alpha 2 codes.
# names get fed into the country dropdowns, whereas their alpha 2 codes get passed into the main function based
# on the corresponding country chosen
def getCountries():
    countries = []
    countries_list = list(pycountry.countries)
    for country in countries_list:
        countries.append([country.name, country.alpha_2])
    return countries

# buildDataframes populates two dataframes for each country chosen based on dropdown choice and country alpha 2 code
# passed by getCountries. buildDataframes also gets a range of years from an IntRangeSlider from interact. 
def buildDataframes(indicator, countryISO_1, countryISO_2, year1, year2):
    dat1 = pd.DataFrame({'A' : []})
    dat2 = pd.DataFrame({'A' : []})
    try:  
        dat1 = wb.download(indicator=indicator, country=[countryISO_1], start=year1, end=year2)
        dat1.columns = [indicator]
        dat2 = wb.download(indicator=indicator, country=[countryISO_2], start=year1, end=year2)
        dat2.columns = [indicator]
    except ValueError:
        display(HTML("<h3>No Data for Indicators Selected</h3>"))
    return dat1, dat2

#dropdowns are initialized using the interact Dropdown function
topic_dropdown = Dropdown(options = getTopics())
indicator_dropdown = Dropdown(options = getIndicators(topic_dropdown.value))
country_name_dropdown1 = Dropdown(options = getCountries())
country_name_dropdown2 = Dropdown(options = getCountries())

#year slider from interact
year_slider = widgets.IntRangeSlider(
        value=[1960, 2020],
        min=1960,
        max=2020,
        step=1,
        description='Year Range:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    )
display(HTML("<h1>The World Bank Country Data Comparison Visualizer</h1>"))

#interact function that takes dropdowns and slider
@interact(topic = topic_dropdown, 
          indicator = indicator_dropdown, 
          country1 = country_name_dropdown1, 
          country2 = country_name_dropdown2, 
          year = year_slider )

def dropdown(country1, country2, topic, indicator, year): #main function
    indicator_dropdown.options = getIndicators(topic_dropdown.value) # passes topic dropdown value to getIndicators
    countryISO_1 = country1                                          # which becomes the options for the indicator dropdown
    countryISO_2 = country2
    year_range =  year_slider.value # gets year values from year_range slider
    year1 = year_slider.value[0] # gets first year 
    year2 = year_slider.value[1] # gets second year
    dat1, dat2 = buildDataframes(indicator, countryISO_1, countryISO_2, year1, year2) # two dataframes are built
    if int(dat1.count()) > 0 and int(dat2.count()) > 0: #error checking making sure that there is data in the dataframes
        dat3 = dat1.dropna() #removes NAN values from the dataframes
        dat4 = dat2.dropna()
        dat3.reset_index(inplace=True) #resets the index on dataframes to allow them to be iterable 
        dat4.reset_index(inplace=True)
        indicator_name = indicators_lookup[indicator] #sets the indicator name based on the indicator list
        dat3.columns = ['Country 1','Year', indicator_name] #names the columns of the dataframes
        dat4.columns = ['Country 2','Year', indicator_name]
        dat = pd.merge(dat3, dat4, on = 'Year') #merges the dataframes on column 'Year' into one for plotly purposes 
        display(dat)
        dat.reset_index(inplace=True) #resets the dataframe index
        try: #error checking to make sure there is graphable data
            fig = px.line(dat, x=dat['Year'], y= [dat.columns[3], dat.columns[5]]) #plots the data on a plotly graph
            fig.update_xaxes(categoryorder='category ascending') #sets years to an ascending value
            fig.show() 
        except:
            display(HTML("<h3>No graphical data for indicators selected, try a different combination</h3>"))
    else:
        display(HTML("<h2>No data found for countries selected, try a different combination</h2>"))
    

interactive(children=(Dropdown(description='country1', options=(['Aruba', 'AW'], ['Afghanistan', 'AF'], ['Ango…

### Prepare for your Pitch and Reflection

With the project code complete, its time to prepare for the final deliverable - submitting your project demo Pitch and reflection.


In [ ]:
# run this code to turn in your work!
from coursetools.submission import Submission
Submission().submit()